In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

In [55]:
data = pd.read_csv('racing_bar.csv')
data.head(2)

,country,code,year,"suicide rate (deaths per 100,000)",color_code,region
0,Afghanistan,AFG,1990,10.318504,#000000,Eastern Mediterranean
1,Afghanistan,AFG,1991,10.327010,#000000,Eastern Mediterranean


In [56]:
data = data.copy().rename(columns={'suicide rate (deaths per 100,000)':'rate'})

In [57]:
years = data['year'].unique()

In [58]:
data

,country,code,year,rate,color_code,region
0,Afghanistan,AFG,1990,10.318504,#000000,Eastern Mediterranean
1,Afghanistan,AFG,1991,10.327010,#000000,Eastern Mediterranean
2,Afghanistan,AFG,1992,10.271411,#000000,Eastern Mediterranean
3,Afghanistan,AFG,1993,10.376123,#000000,Eastern Mediterranean
4,Afghanistan,AFG,1994,10.575915,#000000,Eastern Mediterranean
...,...,...,...,...,...,...
5371,Zimbabwe,ZWE,2013,28.361200,#CC99FF,Africa
5372,Zimbabwe,ZWE,2014,27.605547,#CC99FF,Africa
5373,Zimbabwe,ZWE,2015,27.197061,#CC99FF,Africa
5374,Zimbabwe,ZWE,2016,26.839591,#CC99FF,Africa


In [60]:
# create a dict with keys 1-28 and values as DataFrames of a year and a region
n_frame = {}
for i, year in enumerate(years):
    df = data[(data['year']==year) & (data['region']=='Europe')] 
    df = df.nlargest(n=5, columns=['rate']) # Only top 5 suicide rate values(countries)
    df = df.sort_values(by=['rate'], ascending=True)
    n_frame[i] = df

In [68]:
# Use go to create the animation
trace = go.Bar(
    x=n_frame[0]['rate'],
    y=n_frame[0]['country'],
    orientation='h',
    text=n_frame[0]['rate'],
    texttemplate='%{text:.3s}', # First 
    marker={
        'color':n_frame[0]['color_code']
    }
)
bar_data = [trace]
layout = go.Layout(
    title = {'text':'Top 5 country in suicide rate (Europe)',
             'font':{'size':28}, 'x':0.5, 'xanchor':'center'},
    xaxis = dict(title={'text':'suicide rate','font':{'size':18}},
                 range=[0, 100], autorange=False
                ),
    yaxis=dict(range=[-0.5, 5.5], autorange=False,tickfont=dict(size=14)),

    # Add buttons
    updatemenus=[{
        'type':'buttons',
        'buttons':[{
            'label':"Play",
            'method':'animate',
            'args':[None, 
                    {"frame":{"duration":1000, "redraw":True},
                    "transition":{"duration":250,
                                "easing":"linear"}}]
        }]
    }]
)
frames = [go.Frame(data=[go.Bar(
                        x=value['rate'],
                        y=value['country'],
                        orientation='h',
                        text=value['rate'],
                        texttemplate='%{text:.3s}', # First 
                        marker={
                            'color':value['color_code']
                        }
                    )],
                layout=go.Layout(
                        title = {'text':'Top 5 country in suicide rate (Europe)'+str(value['year'].values[0]),
                                'font':{'size':28}, 'x':0.5, 'xanchor':'center'},
                        xaxis = dict(title={'text':'suicide rate','font':{'size':18}},
                                    range=[0, 100], autorange=False
                                    ),
                        yaxis=dict(range=[-0.5, 5.5], autorange=False,tickfont=dict(size=14)),
                    ))
          for key, value in n_frame.items()]

fig = go.Figure(data=bar_data, layout=layout, frames=frames)
pio.show(fig)